In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
import math
import torch

In [2]:
data = pandas.read_csv('../result/Power_10000_ptanh_zero_cross_pNN_ACT_interactive1.csv',nrows= 0)
data = data.iloc[:,1:]

In [3]:
datalist=data.columns.values.tolist()
print(datalist)
print(len(datalist))

['/V2/PLUS R1_data 784293.4 R2_data 25777.58 W1_data 8.26e-05 L1_data 7.57e-05 W2_data 0.000481804 L2_data 3e-05  (A)', '/V2/PLUS R1_data 876822.7 R2_data 24576.26 W1_data 9.21e-05 L1_data 4.87e-05 W2_data 0.000482482 L2_data 3e-05  (A)', '/V2/PLUS R1_data 1869442 R2_data 21271.33 W1_data 9.42e-05 L1_data 5.73e-05 W2_data 0.000482596 L2_data 3e-05  (A)', '/V2/PLUS R1_data 541449.1 R2_data 23507.55 W1_data 8.82e-05 L1_data 6.86e-05 W2_data 0.000482647 L2_data 3e-05  (A)', '/V2/PLUS R1_data 698784.9 R2_data 15652.17 W1_data 9.5e-05 L1_data 5.83e-05 W2_data 0.000482719 L2_data 3e-05  (A)', '/V2/PLUS R1_data 319306.9 R2_data 24370.84 W1_data 9.03e-05 L1_data 5.27e-05 W2_data 0.000482948 L2_data 3e-05  (A)', '/V2/PLUS R1_data 1300225 R2_data 17196.72 W1_data 9.53e-05 L1_data 7.78e-05 W2_data 0.000483089 L2_data 3e-05  (A)', '/V2/PLUS R1_data 920387 R2_data 17183.72 W1_data 9.91e-05 L1_data 7.61e-05 W2_data 0.000483113 L2_data 3e-05  (A)', '/V2/PLUS R1_data 1156031 R2_data 17456.75 W1_data 8

In [8]:
result = []
for j in range(10000):
    a = datalist[j].split(' ')
    nums = []
    for k in range(2, 13, 2):
        
        nums.append(a[k])

    b=map(float,nums)    
    result.append(list(b))

In [9]:
result_temp = torch.tensor(result)
result_temp.max(0)[0]

tensor([2.0500e+06, 3.2000e+04, 1.0000e-04, 8.0000e-05, 5.0200e-04, 4.0000e-05])

In [10]:
result_temp.min(0)[0]

tensor([2.5000e+05, 6.0000e+03, 8.0000e-05, 4.0000e-05, 4.8000e-04, 3.0000e-05])

In [11]:
c = torch.tensor(result)
c.shape

torch.Size([10000, 6])

In [12]:
np.savetxt('./data/act_RT.csv', c, fmt='%f', delimiter=',')

In [13]:
r1 = c[:, 0] * 1
r2 = c[:, 1] * 1
ratio_1 = r2/r1
w1 = c[:, 2] * 1
l1 = c[:, 3] * 1
ratio_2 = l1/w1
w2 = c[:, 4] * 1
l2 = c[:, 5] * 1
ratio_3 = l2/w2

In [14]:
a = torch.hstack((c, ratio_1.reshape(-1, 1),ratio_2.reshape(-1, 1), ratio_3.reshape(-1, 1)))
a.shape

torch.Size([10000, 9])

In [15]:
np.savetxt('./data/act_RT_ratio.csv', a, fmt='%f', delimiter=',')